## 1. Import python libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm import tqdm #progress indicator

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None


## 2. Import main dataset using API

In [4]:
def import_dataset_api (url):

    params = {
        'limit': 1,
        'offset': 0,
        'count': 'true',
        'results': 'false',
        'format': 'json',    }
        
    response = requests.get(url, params=params, timeout=10)
    data = response.json()

    total_rows = data['count']
    print(f"Total rows to fetch: {total_rows}")

    params = {
        'limit': 500, #Limit on https://data.cms.gov/
        'offset': 0,        
        'count': 'false',   
        'results': 'true',  
        'format': 'json',   
    }

    all_results = []

    with tqdm(total=total_rows) as pbar:
        while True:
            response = requests.get(url, params=params)
            data = response.json()

            #Add the obtained data to the array. Data is under the ‘results’ key
            results = data['results']
            all_results.extend(results)

            pbar.update(len(results))

            if len(results) < params['limit']:
                break

            params['offset'] += params['limit']

    df = pd.DataFrame(all_results)
    print(f"Number of rows after fetching: {df.shape[0]}")
    
    return df


In [5]:
api_url = "https://data.cms.gov/provider-data/api/1/datastore/query/77hc-ibv8/0"

df = import_dataset_api(api_url)


Total rows to fetch: 172044


100%|█████████████████████████████████████████████████████████████████████████| 172044/172044 [07:04<00:00, 405.22it/s]


Number of rows after fetching: 172044


In [6]:
print(f'Dataset has {df.shape[0]} rows and {df.shape[1]} columns.')
display(df.dtypes, df.head(5))

Dataset has 172044 rows and 15 columns.


facility_id             object
facility_name           object
address                 object
citytown                object
state                   object
zip_code                object
countyparish            object
telephone_number        object
measure_id              object
measure_name            object
compared_to_national    object
score                   object
footnote                object
start_date              object
end_date                object
dtype: object

,facility_id,facility_name,address,citytown,state,zip_code,countyparish,telephone_number,measure_id,measure_name,compared_to_national,score,footnote,start_date,end_date
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.182,,01/01/2023,12/31/2023
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.099,,01/01/2023,12/31/2023
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,9538,,01/01/2023,12/31/2023
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,10.082,,01/01/2023,12/31/2023
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,,01/01/2023,12/31/2023


## 3. Data understanding

In [7]:
#Number of hospitals in dataset
len(df.facility_id.unique())

4779

Duplicate data identification:

In [8]:
df.duplicated().sum()

0

Duplicate data identification with subsets:

In [9]:
#Verify Duplicate Measures per Hospital

subset_check_analysis_data = ['facility_id', 'facility_name', 'measure_id']
df.duplicated(subset = subset_check_analysis_data).sum()

0

In [10]:
#Verify Hospital Contact Information Consistency

"""
Check if each hospital (4000+ unique facilities) maintains consistent contact details across all entries.
If data is consistent, the number of duplicated rows should equal total rows minus number of hospitals.
"""

hospitals_count = len(df.facility_id.unique())

subset_check_hospital_data = ['facility_id', 'facility_name', 'address', 'citytown', 'state', 'zip_code', 'countyparish', 
               'telephone_number']
hospital_contact_data_duplicates = df.duplicated(subset = subset_check_hospital_data).sum()


if (df.shape[0] - hospital_contact_data_duplicates) == hospitals_count:
    print(f"Contact data is correct. Number of duplicate records (according to the subset): {hospital_contact_data_duplicates}")
else:
    print(f"Warning: Some hospitals have different contact details in different records. Consider whether this may affect your analysis.")
    


Contact data is correct. Number of duplicate records (according to the subset): 167265


Missing values:

In [11]:
df.isna().sum()

facility_id             0
facility_name           0
address                 0
citytown                0
state                   0
zip_code                0
countyparish            0
telephone_number        0
measure_id              0
measure_name            0
compared_to_national    0
score                   0
footnote                0
start_date              0
end_date                0
dtype: int64

In [12]:
"""
After initial check showing 0 missing values, performing additional verification of null values:
- Check unique values
- Review random samples across columns

The contact details of the hospitals were omitted as they had already been checked. 
"""

df.state.unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VT', 'VI', 'VA', 'WA', 'WV', 'WI', 'WY', 'AS', 'GU',
       'MP'], dtype=object)

In [13]:
df.measure_id.unique().tolist()
df.measure_name.unique().tolist()
len(df.measure_id.unique()) == len(df.measure_name.unique())

True

In [14]:
df.compared_to_national.unique().tolist()

['No Different than National Benchmark',
 'Better than the National Benchmark',
 'Not Available',
 'Worse than the National Benchmark']

In [15]:
#Column compared_to_national contains 'Not Available' values, which represent missing data

len(df[df.compared_to_national == 'Not Available'])

103806

In [16]:
#Verify score column data type consistency

df.score.sample(n=10)

116914                2
103940            13549
88797     Not Available
126982                2
127822                1
7901      Not Available
127820               43
75249            15.307
50645     Not Available
135625    Not Available
Name: score, dtype: object

In [17]:
len(df[df.score == 'Not Available'])

73096

In [18]:
"""
Analyze missing data in score column:
- Checking if 'score' column contains another text values as expected
- Counting all missing values
"""

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

str_values_in_score_column = df[df['score'].apply(lambda x: isinstance(x, str) and x != 'Not Available' and not is_float(x))]['score']
str_values_in_score_column.unique().tolist()

['N/A']

In [19]:
"""
Found ~43% missing values. Next steps:
- Identify patterns and causes
- Analyze relationships with other variables
"""


print(f"Column score consist {len(df[df.score == 'Not Available']) + len(str_values_in_score_column)}")

Column score consist 75027


In [20]:
#Checking footnote column. Source - https://data.cms.gov/provider-data/dataset/y9us-9xdf

"""
3 - Results are based on a shorter time period than required.
5 - Results are not available for this reporting period.
8 - The lower limit of the confidence interval cannot be calculated if the number of observed infections equals zero.
12 - This measure does not apply to this hospital for this reporting period.
13 - Results cannot be calculated for this reporting period.
19 - Data are shown only for hospitals that participate in the Inpatient Quality Reporting (IQR) and Outpatient Quality Reporting (OQR) programs.
28 - The results are based on the hospital or facility's data submissions. CMS approved the hospital or facility's Extraordinary Circumstances Exception request suggesting that results may be impacted.
"""

sorted(df.footnote.unique().tolist())


['', '12', '13', '13, 28', '19', '28', '3', '3, 13', '5', '8', '8, 28']

In [21]:
"""
Analyze missing data patterns in score column with footnotes

1. Missing values are either 'Not Available' or 'N/A' with explanatory footnotes
2. All 75027 missing values in score column correspond to specific footnotes explaining data limitations
"""

len(df[((df.score == 'Not Available')|(df.score == 'N/A')) & (df.footnote != '')])

75027

In [22]:
"""
Found 1927 rows with numerical scores but with footnotes indicating limitations (3, 28)

Filtering decisions based on analysis:

1. Remove rows with missing scores ('Not Available', 'N/A') as they lack analytical value
2. Retain rows with footnotes 3 or 28 where measures were calculated: <br>
    Footnote 3: Presence of calculated measures indicates sufficient sample size, despite shorter time period <br>
    Footnote 28: Administrative note, doesn't impact data quality<br>
"""

df[((df.score != 'Not Available')&(df.score != 'N/A')) & (df.footnote != '')]['footnote'].unique().tolist()
len(df[((df.score != 'Not Available')&(df.score != 'N/A')) & (df.footnote != '')])


1927

<br>
Data restructuring needs based on analysis:

Current structure has limitations:
1. Multiple rows per hospital (36 different measure categories)
2. Score (measure_id, measure_name) column contains mixed indicators, no analysis possible without filtering the required data every single time
3. Redundant contact information across rows

Plan:
- Reshape dataset: pivot measures into columns
- Eliminate redundant hospital information
- Remove unnecessary columns
- Convert columns to appropriate data types
- Delete rows, where HAI_6_NUMERATOR and HAI_6_SIR values are missing as they lack analytical value

- Import additional table - Hospital General Information and Healthcare associated infections by state
- Prepate additional tables to analysis



## 4. Data cleaning

In [23]:
df_clean = df.copy()

In [24]:
df_clean = df_clean.drop(columns = ['footnote'], axis = 1)

In [25]:
#Choosing only Clostridium difficile (C.diff.) data

df_hai_6_data = df_clean[df_clean['measure_id'].str.startswith('HAI_6_')]

df_pivot = df_hai_6_data.pivot(index=['facility_id', 'facility_name', 'state'],
                                      columns='measure_id',
                                      values='score').reset_index()

df_pivot.columns.name = None

In [26]:
#Adding compared_to_national column

metric_groups = {
    'HAI_6': ['HAI_6_CILOWER', 'HAI_6_CIUPPER', 'HAI_6_DOPC', 'HAI_6_ELIGCASES', 'HAI_6_NUMERATOR', 'HAI_6_SIR']
}

for group_name, measure_ids in metric_groups.items():
    # Inicjalizuj kolumnę `compared_to_national` dla tej grupy
    df_pivot[f'{group_name}_compared_to_national'] = None
    
    # Przypisz wartości `compared_to_national` dla danej grupy metryk
    for i, row in df_clean.iterrows():
        facility_id = row['facility_id']
        
        # Jeśli `measure_id` jest częścią grupy, przypisz `compared_to_national` tylko raz dla całej grupy
        if row['measure_id'] in measure_ids:
            if pd.isna(df_pivot.loc[df_pivot['facility_id'] == facility_id, f'{group_name}_compared_to_national']).all():
                df_pivot.loc[df_pivot['facility_id'] == facility_id, f'{group_name}_compared_to_national'] = row['compared_to_national']


In [27]:
#Change 'Not Available' and 'N/A' values for future analysis purpose

df_pivot.replace({'Not Available': np.nan, 'N/A': np.nan}, inplace=True)

In [28]:
df_pivot.isna().sum()

facility_id                      0
facility_name                    0
state                            0
HAI_6_CILOWER                 2231
HAI_6_CIUPPER                 1806
HAI_6_DOPC                     693
HAI_6_ELIGCASES                695
HAI_6_NUMERATOR                695
HAI_6_SIR                     1806
HAI_6_compared_to_national    1806
dtype: int64

In [29]:
def change_data_type(df, columns, data_type):
    for column in columns:
        df[column] = df[column].astype(data_type)
    
    return df

In [30]:
df_pivot = change_data_type(df_pivot, ['HAI_6_DOPC', 'HAI_6_NUMERATOR'], 'Int64')
df_pivot = change_data_type(df_pivot, ['HAI_6_SIR', 'HAI_6_ELIGCASES'], float)

In [31]:
df_pivot.sample(n = 5)

,facility_id,facility_name,state,HAI_6_CILOWER,HAI_6_CIUPPER,HAI_6_DOPC,HAI_6_ELIGCASES,HAI_6_NUMERATOR,HAI_6_SIR,HAI_6_compared_to_national
4392,490145,STONESPRINGS HOSPITAL CENTER,VA,NaN,2.116,6981,1.416,0,0.000,No Different than National Benchmark
1475,161306,JONES REGIONAL MEDICAL CENTER,IA,NaN,NaN,1329,0.435,0,NaN,NaN
3772,430015,AVERA ST MARY'S HOSPITAL,SD,0.384,2.912,6596,3.313,4,1.207,No Different than National Benchmark
2742,310041,COMMUNITY MEDICAL CENTER,NJ,0.100,0.287,120111,79.768,14,0.176,Better than the National Benchmark
4181,451335,MUENSTER MEMORIAL HOSPITAL,TX,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN


In [32]:
df_pivot[(df_pivot.HAI_6_ELIGCASES < 1) & (df_pivot.HAI_6_CIUPPER.isna())].sample(n=3)

,facility_id,facility_name,state,HAI_6_CILOWER,HAI_6_CIUPPER,HAI_6_DOPC,HAI_6_ELIGCASES,HAI_6_NUMERATOR,HAI_6_SIR,HAI_6_compared_to_national
2242,241328,SANFORD BAGLEY MEDICAL CENTER,MN,NaN,NaN,1928,0.426,0,NaN,NaN
1865,191305,ST JAMES PARISH HOSPITAL,LA,NaN,NaN,1349,0.457,0,NaN,NaN
3491,390025,KENSINGTON HOSPITAL,PA,NaN,NaN,1956,0.320,0,NaN,NaN


In [33]:
#Drop rows where HAI_6_NUMERATOR value is missing

df_pivot = df_pivot[~df_pivot['HAI_6_NUMERATOR'].isnull()]
df_pivot = df_pivot[~df_pivot['HAI_6_SIR'].isnull()]

In [34]:
#Drop columns from dataset that are not needed for analysis

df_drop_columns = ['HAI_6_CILOWER', 'HAI_6_CIUPPER']
df_pivot = df_pivot.drop(df_drop_columns, axis = 1)

In [35]:
df_pivot

,facility_id,facility_name,state,HAI_6_DOPC,HAI_6_ELIGCASES,HAI_6_NUMERATOR,HAI_6_SIR,HAI_6_compared_to_national
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,AL,103195,65.234,32,0.491,Better than the National Benchmark
1,010005,MARSHALL MEDICAL CENTERS,AL,33872,9.503,6,0.631,No Different than National Benchmark
2,010006,NORTH ALABAMA MEDICAL CENTER,AL,64728,37.771,1,0.026,Better than the National Benchmark
3,010007,MIZELL MEMORIAL HOSPITAL,AL,5267,1.994,2,1.003,No Different than National Benchmark
5,010011,ST. VINCENT'S EAST,AL,80106,42.140,21,0.498,Better than the National Benchmark
...,...,...,...,...,...,...,...,...
4755,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER PFLUGERVILLE,TX,5463,1.157,0,0.000,No Different than National Benchmark
4757,670132,METHODIST SOUTHLAKE MEDICAL CENTER,TX,2914,1.194,0,0.000,No Different than National Benchmark
4764,670260,TEXAS HEALTH HOSPITAL FRISCO,TX,19411,9.659,2,0.207,Better than the National Benchmark
4770,670300,METHODIST MIDLOTHIAN MEDICAL CENTER,TX,10294,4.995,1,0.200,Better than the National Benchmark


In [36]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2973 entries, 0 to 4771
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   facility_id                 2973 non-null   object 
 1   facility_name               2973 non-null   object 
 2   state                       2973 non-null   object 
 3   HAI_6_DOPC                  2973 non-null   Int64  
 4   HAI_6_ELIGCASES             2973 non-null   float64
 5   HAI_6_NUMERATOR             2973 non-null   Int64  
 6   HAI_6_SIR                   2973 non-null   float64
 7   HAI_6_compared_to_national  2973 non-null   object 
dtypes: Int64(2), float64(2), object(4)
memory usage: 214.8+ KB


In [ ]:
"""

# Identyfikacja wartości odstających
q1 = df_pivot['HAI_6_NUMERATOR'].quantile(0.25)
q3 = df_pivot['HAI_6_NUMERATOR'].quantile(0.75)
iqr = q3 - q1
outliers = df_pivot[(df_pivot['HAI_6_NUMERATOR'] < q1 - 1.5*iqr) | (df_pivot['HAI_6_NUMERATOR'] > q3 + 1.5*iqr)]
print("Liczba wartości odstających:", len(outliers))
print(outliers.HAI_6_NUMERATOR.tolist())


"""

## 5. Import editional datasets

### Healthcare Associated Infections - State

In [37]:
df_state_infections = import_dataset_api("https://data.cms.gov/provider-data/api/1/datastore/query/k2ze-bqvw/0")

Total rows to fetch: 1008


100%|█████████████████████████████████████████████████████████████████████████████| 1008/1008 [00:03<00:00, 314.58it/s]

Number of rows after fetching: 1008


In [38]:
df_state_infections = df_state_infections.pivot(index='state',
                                      columns='measure_id',
                                      values='score').reset_index()

df_state_infections.columns.name = None

In [39]:
state_infections_drop_columns = ['HAI_6_CILOWER', 'HAI_6_CIUPPER',
                                'HAI_5_CILOWER', 'HAI_5_CIUPPER',
                                'HAI_4_CILOWER', 'HAI_4_CIUPPER',
                                'HAI_3_CILOWER', 'HAI_3_CIUPPER',
                                'HAI_2_CILOWER', 'HAI_2_CIUPPER',
                                'HAI_1_CILOWER', 'HAI_1_CIUPPER']

df_state_infections = df_state_infections.drop(state_infections_drop_columns, axis = 1)

In [40]:
df_state_infections.replace({'Not Available': np.nan}, inplace = True)

In [41]:
df_state_infections = change_data_type(df_state_infections,
                                      ['HAI_1_SIR', 'HAI_2_SIR', 'HAI_3_SIR', 'HAI_4_SIR', 'HAI_5_SIR', 'HAI_6_SIR'],
                                      float)

In [42]:
df_state_infections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   state      56 non-null     object 
 1   HAI_1_SIR  52 non-null     float64
 2   HAI_2_SIR  52 non-null     float64
 3   HAI_3_SIR  51 non-null     float64
 4   HAI_4_SIR  52 non-null     float64
 5   HAI_5_SIR  52 non-null     float64
 6   HAI_6_SIR  52 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.2+ KB


In [43]:
df_state_infections.sample(n = 10)

,state,HAI_1_SIR,HAI_2_SIR,HAI_3_SIR,HAI_4_SIR,HAI_5_SIR,HAI_6_SIR
42,PR,2.969,0.971,NaN,2.385,0.164,0.393
9,DE,0.748,0.614,0.785,0.766,0.641,0.313
22,MD,0.833,0.681,1.011,1.796,0.575,0.502
27,MP,NaN,NaN,NaN,NaN,NaN,NaN
45,SD,0.688,0.881,0.670,0.812,0.498,0.482
17,IN,0.530,0.551,0.812,1.377,0.698,0.410
50,VI,NaN,NaN,NaN,NaN,NaN,NaN
5,CA,0.753,0.707,0.878,0.901,0.717,0.481
12,GU,NaN,NaN,NaN,NaN,NaN,NaN
52,WA,0.705,0.794,1.038,1.276,0.614,0.554


In [44]:
df_state_infections = df_state_infections[~df_state_infections['HAI_6_SIR'].isnull()].reset_index()

In [45]:
df_state_infections = df_state_infections.drop('index', axis = 1)

### Hospital General Information

In [46]:
df_hospitals = import_dataset_api("https://data.cms.gov/provider-data/api/1/datastore/query/xubh-q36u/0")

Total rows to fetch: 5394


100%|█████████████████████████████████████████████████████████████████████████████| 5394/5394 [00:13<00:00, 386.71it/s]

Number of rows after fetching: 5394


In [47]:
df_hospitals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5394 entries, 0 to 5393
Data columns (total 39 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   facility_id                                            5394 non-null   object
 1   facility_name                                          5394 non-null   object
 2   address                                                5394 non-null   object
 3   citytown                                               5394 non-null   object
 4   state                                                  5394 non-null   object
 5   zip_code                                               5394 non-null   object
 6   countyparish                                           5394 non-null   object
 7   telephone_number                                       5394 non-null   object
 8   hospital_type                                          539

In [48]:
#Deletion of data on hospitals not included in the main table

df_hospitals = df_hospitals[df_hospitals['facility_id'].isin(df_pivot['facility_id'])]

In [49]:
"""
After reviewing the data dictionary of this dataset, the following columns will be removed 
because they do not have relevant information for the analysis, or they are duplicates of other data.
"""

hospitals_drop_columns = ['meets_criteria_for_promoting_interoperability_of_ehrs',
                         'meets_criteria_for_birthing_friendly_designation',
                         'pt_exp_group_measure_count',
                         'count_of_facility_pt_exp_measures',
                         'pt_exp_group_footnote',
                         'te_group_measure_count',
                         'count_of_facility_te_measures',
                         'te_group_footnote',
                         'facility_name',
                         'address',
                         'citytown',
                         'state',
                         'zip_code',
                         'countyparish',
                         'telephone_number',
                         'hospital_overall_rating_footnote',
                         'mort_group_footnote',
                         'safety_group_footnote',
                         'readm_group_footnote']

df_hospitals = df_hospitals.drop(hospitals_drop_columns, axis = 1)

In [50]:
df_hospitals.replace({'Not Available': np.nan}, inplace = True)

In [51]:
hospitals_columns_to_convert = ['hospital_overall_rating', 'mort_group_measure_count',
                     'count_of_facility_mort_measures', 'count_of_mort_measures_better',
                     'count_of_mort_measures_no_different', 'count_of_mort_measures_worse',
                     'safety_group_measure_count', 'count_of_facility_safety_measures',
                     'count_of_safety_measures_better', 'count_of_safety_measures_no_different',
                     'count_of_safety_measures_worse', 'readm_group_measure_count',
                     'count_of_readm_measures_better', 'count_of_readm_measures_no_different',
                     'count_of_readm_measures_worse', 'count_of_facility_readm_measures']

df_hospitals = change_data_type(df_hospitals, hospitals_columns_to_convert, 'Int64')

In [52]:
df_hospitals.isna().sum()

facility_id                                0
hospital_type                              0
hospital_ownership                         0
emergency_services                         0
hospital_overall_rating                  367
mort_group_measure_count                  19
count_of_facility_mort_measures          179
count_of_mort_measures_better            179
count_of_mort_measures_no_different      179
count_of_mort_measures_worse             179
safety_group_measure_count                19
count_of_facility_safety_measures         59
count_of_safety_measures_better           59
count_of_safety_measures_no_different     59
count_of_safety_measures_worse            59
readm_group_measure_count                 19
count_of_facility_readm_measures          40
count_of_readm_measures_better            40
count_of_readm_measures_no_different      40
count_of_readm_measures_worse             40
dtype: int64

In [53]:
df_hospitals.dtypes

facility_id                              object
hospital_type                            object
hospital_ownership                       object
emergency_services                       object
hospital_overall_rating                   Int64
mort_group_measure_count                  Int64
count_of_facility_mort_measures           Int64
count_of_mort_measures_better             Int64
count_of_mort_measures_no_different       Int64
count_of_mort_measures_worse              Int64
safety_group_measure_count                Int64
count_of_facility_safety_measures         Int64
count_of_safety_measures_better           Int64
count_of_safety_measures_no_different     Int64
count_of_safety_measures_worse            Int64
readm_group_measure_count                 Int64
count_of_facility_readm_measures          Int64
count_of_readm_measures_better            Int64
count_of_readm_measures_no_different      Int64
count_of_readm_measures_worse             Int64
dtype: object

In [54]:
df_hospitals['hospital_type'].unique().tolist()

['Acute Care Hospitals',
 'Critical Access Hospitals',
 'Acute Care - Department of Defense']

In [55]:
df_hospitals['hospital_ownership'].unique().tolist()

['Government - Hospital District or Authority',
 'Proprietary',
 'Voluntary non-profit - Private',
 'Government - State',
 'Voluntary non-profit - Other',
 'Government - Local',
 'Voluntary non-profit - Church',
 'Government - Federal',
 'Tribal',
 'Department of Defense',
 'Physician']

## 6. Join tables

In [56]:
df_final = df_pivot.merge(df_hospitals, on='facility_id', how = 'left')

In [57]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   facility_id                            2973 non-null   object 
 1   facility_name                          2973 non-null   object 
 2   state                                  2973 non-null   object 
 3   HAI_6_DOPC                             2973 non-null   Int64  
 4   HAI_6_ELIGCASES                        2973 non-null   float64
 5   HAI_6_NUMERATOR                        2973 non-null   Int64  
 6   HAI_6_SIR                              2973 non-null   float64
 7   HAI_6_compared_to_national             2973 non-null   object 
 8   hospital_type                          2973 non-null   object 
 9   hospital_ownership                     2973 non-null   object 
 10  emergency_services                     2973 non-null   object 
 11  hosp

In [58]:
categorical_columns = ['emergency_services', 'hospital_ownership', 'hospital_type', 
                       'HAI_6_compared_to_national','state']

df_final = change_data_type(df_final, categorical_columns, 'category')

In [59]:
df_final.columns = df_final.columns.str.lower()

In [60]:
df_final.dtypes

facility_id                                object
facility_name                              object
state                                    category
hai_6_dopc                                  Int64
hai_6_eligcases                           float64
hai_6_numerator                             Int64
hai_6_sir                                 float64
hai_6_compared_to_national               category
hospital_type                            category
hospital_ownership                       category
emergency_services                       category
hospital_overall_rating                     Int64
mort_group_measure_count                    Int64
count_of_facility_mort_measures             Int64
count_of_mort_measures_better               Int64
count_of_mort_measures_no_different         Int64
count_of_mort_measures_worse                Int64
safety_group_measure_count                  Int64
count_of_facility_safety_measures           Int64
count_of_safety_measures_better             Int64


## 7. Data Analysis

In [61]:
df_final

,facility_id,facility_name,state,hai_6_dopc,hai_6_eligcases,hai_6_numerator,hai_6_sir,hai_6_compared_to_national,hospital_type,hospital_ownership,emergency_services,hospital_overall_rating,mort_group_measure_count,count_of_facility_mort_measures,count_of_mort_measures_better,count_of_mort_measures_no_different,count_of_mort_measures_worse,safety_group_measure_count,count_of_facility_safety_measures,count_of_safety_measures_better,count_of_safety_measures_no_different,count_of_safety_measures_worse,readm_group_measure_count,count_of_facility_readm_measures,count_of_readm_measures_better,count_of_readm_measures_no_different,count_of_readm_measures_worse
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,AL,103195,65.234,32,0.491,Better than the National Benchmark,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3,7,7,1,6,0,8,7,2,5,0,11,11,1,8,2
1,010005,MARSHALL MEDICAL CENTERS,AL,33872,9.503,6,0.631,No Different than National Benchmark,Acute Care Hospitals,Government - Hospital District or Authority,Yes,2,7,6,0,5,1,8,7,0,7,0,11,9,0,8,1
2,010006,NORTH ALABAMA MEDICAL CENTER,AL,64728,37.771,1,0.026,Better than the National Benchmark,Acute Care Hospitals,Proprietary,Yes,1,7,7,0,6,1,8,7,3,4,0,11,9,0,7,2
3,010007,MIZELL MEMORIAL HOSPITAL,AL,5267,1.994,2,1.003,No Different than National Benchmark,Acute Care Hospitals,Voluntary non-profit - Private,Yes,1,7,3,0,2,1,8,2,0,2,0,11,7,0,7,0
4,010011,ST. VINCENT'S EAST,AL,80106,42.140,21,0.498,Better than the National Benchmark,Acute Care Hospitals,Voluntary non-profit - Private,Yes,3,7,7,0,7,0,8,7,1,6,0,11,9,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER PFLUGERVILLE,TX,5463,1.157,0,0.000,No Different than National Benchmark,Acute Care Hospitals,Voluntary non-profit - Private,Yes,<NA>,7,1,0,1,0,8,2,0,2,0,11,5,0,5,0
2969,670132,METHODIST SOUTHLAKE MEDICAL CENTER,TX,2914,1.194,0,0.000,No Different than National Benchmark,Acute Care Hospitals,Proprietary,Yes,<NA>,7,<NA>,<NA>,<NA>,<NA>,8,2,0,2,0,11,3,0,3,0
2970,670260,TEXAS HEALTH HOSPITAL FRISCO,TX,19411,9.659,2,0.207,Better than the National Benchmark,Acute Care Hospitals,Voluntary non-profit - Private,Yes,3,7,2,0,2,0,8,5,0,5,0,11,5,1,4,0
2971,670300,METHODIST MIDLOTHIAN MEDICAL CENTER,TX,10294,4.995,1,0.200,Better than the National Benchmark,Acute Care Hospitals,Voluntary non-profit - Private,Yes,4,7,<NA>,<NA>,<NA>,<NA>,8,3,0,3,0,11,3,0,3,0


##### How many infections have been reported?

In [62]:
total_cases = df_final['hai_6_numerator'].sum()
total_cases

34523

##### What is the number of hospitals representing each type in the dataset?

<strong>Acute Care - Department of Defence</strong>: Acute care hospitals owned by the U.S. Department of Defence typically serve military personnel and their families. They focus on providing emergency care for injuries, infections and other acute conditions.<br><br>
<strong>Acute Care Hospitals</strong>: General acute care hospitals that provide intensive treatment for illness and injury. They can accommodate any patient group and provide a wide range of healthcare services, including surgery and emergency care.<br><br>
<strong>Critical Access Hospitals</strong>: Hospitals with limited access, usually located in rural areas where access to healthcare is difficult. They provide basic medical care, and often have limited capacity for emergency care.

In [63]:
df_final.groupby('hospital_type')['facility_id'].count().sort_values(ascending = False)

hospital_type
Acute Care Hospitals                  2681
Critical Access Hospitals              276
Acute Care - Department of Defense      16
Name: facility_id, dtype: int64

In [64]:
#Percentage of hospitals of each type

total_hospitals = df_final['facility_id'].nunique()
hospitals_pct = round((df_final.groupby('hospital_type')['facility_id'].count() / total_hospitals * 100), 2)
hospitals_pct.sort_values(ascending = False)

hospital_type
Acute Care Hospitals                  90.18
Critical Access Hospitals              9.28
Acute Care - Department of Defense     0.54
Name: facility_id, dtype: float64

##### What is the number of cases per hospital type?

In [65]:
df_final.groupby('hospital_type')['hai_6_numerator'].sum().sort_values(ascending = False)

hospital_type
Acute Care Hospitals                  34041
Critical Access Hospitals               332
Acute Care - Department of Defense      150
Name: hai_6_numerator, dtype: Int64

##### What is the average number of cases per hospital for each type?

In [66]:
df_final.groupby('hospital_type').agg({
    'hai_6_numerator': 'sum',
    'facility_id': 'count'
}).rename(columns={
    'facility_id': 'Number of hospitals',
    'hai_6_numerator': 'Number of observations',
}).assign(**{
    'Average cases per hospital':lambda x: round(x['Number of observations'] / x['Number of hospitals'], 2)
    })

,Number of observations,Number of hospitals,Average cases per hospital
hospital_type,,,
Acute Care - Department of Defense,150,16,9.38
Acute Care Hospitals,34041,2681,12.7
Critical Access Hospitals,332,276,1.2


Based on the presented results, we can draw the following conclusions: <br>

1. The highest average number of cases per hospital is observed in Acute Care Hospitals - around 12.7 cases per facility. This is also the largest group of hospitals (2681 facilities) with the highest total number of cases (34041).<br>
2. Department of Defense (DoD) hospitals have the second highest average - 9.4 cases per hospital. However, this is the smallest group (only 16 facilities), which may affect the representativeness of this average. <br>
3. Critical Access Hospitals have a significantly lower average - around 1.2 cases per hospital. Despite there being more of them than DoD hospitals (276 facilities), they report proportionally much fewer cases.<br><br>

The differences may be due to the size of the facilities - Acute Care Hospitals are likely larger units serving more patients. Critical Access Hospitals are often located in smaller communities and serve a smaller population, which may explain the lower average.

In [67]:
df_final[df_final.hospital_type == 'Acute Care - Department of Defense']

,facility_id,facility_name,state,hai_6_dopc,hai_6_eligcases,hai_6_numerator,hai_6_sir,hai_6_compared_to_national,hospital_type,hospital_ownership,emergency_services,hospital_overall_rating,mort_group_measure_count,count_of_facility_mort_measures,count_of_mort_measures_better,count_of_mort_measures_no_different,count_of_mort_measures_worse,safety_group_measure_count,count_of_facility_safety_measures,count_of_safety_measures_better,count_of_safety_measures_no_different,count_of_safety_measures_worse,readm_group_measure_count,count_of_facility_readm_measures,count_of_readm_measures_better,count_of_readm_measures_no_different,count_of_readm_measures_worse
67,02013F,673rd Medical Group (Joint Base Elmendorf-Rich...,AK,3965,1.393,1,0.718,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
238,05015F,60th Medical Group (Travis AFB),CA,7973,2.761,8,2.898,Worse than the National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
256,05022F,NMC San Diego,CA,22293,10.940,8,0.731,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
771,11032F,Martin ACH (FT Benning),GA,4234,1.010,1,0.990,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
772,11033F,Dwight Eisenhower AMC (FT Gordon),GA,5630,1.098,0,0.000,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
794,12001F,Tripler AMC (FT Shafter),HI,26822,8.751,10,1.143,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1273,21007F,Walter Reed National Military Med Cen,MD,31429,12.140,17,1.400,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1638,29001F,99th Medical Group (Nellis AFB),NV,8698,5.232,3,0.573,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1939,34011F,NMC Camp Lejeune,NC,13401,4.868,2,0.411,No Different than National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1953,34014F,Womack AMC (FT Bragg),NC,98414,38.137,3,0.079,Better than the National Benchmark,Acute Care - Department of Defense,Department of Defense,Yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


It is not possible to conduct a detailed analysis of the Department of Defence hospitals because the data is not available.

##### How many hospitals have a lower score compared to the national results?


In [68]:
number_of_hospitals_with_worse_score = len(df_final[df_final.hai_6_compared_to_national == 
                                                    'Worse than the National Benchmark'])

number_of_hospitals_with_worse_score

16

In [69]:
round((number_of_hospitals_with_worse_score / total_hospitals * 100), 2)

0.54

Only 16 hospitals are rated worse than the national benchmark, representing less then 1% of all observations.<br>

##### How many hospitals of each type are rated worse, no difference, and better than the national results?

In [70]:
df_final.pivot_table(index=['hospital_type'], columns=['hai_6_compared_to_national'], values=['facility_id'], 
                     aggfunc='count').style.format('{:.0f}')

The majority of Acute Care Hospitals (1704) are rated as better than the national results, while a smaller number (9) are rated worse. Most Critical Access Hospitals (270) are rated as no different than the national benchmark, with 5 rated as worse and only 1 hospital rated better the national benchmark. In no category is the number of worst performing hospitals the leading indicator.

##### What is the number of cases per hospital ownership?

In [71]:
df_final.groupby('hospital_ownership')['hai_6_numerator'].sum().sort_values(ascending = False)

hospital_ownership
Voluntary non-profit - Private                 20159
Government - Hospital District or Authority     3012
Voluntary non-profit - Other                    2911
Proprietary                                     2776
Voluntary non-profit - Church                   2538
Government - State                              1495
Government - Local                              1347
Department of Defense                            150
Physician                                         59
Tribal                                            43
Government - Federal                              33
Name: hai_6_numerator, dtype: Int64

In [72]:
df_final.groupby('hospital_ownership').agg({
    'hai_6_numerator': 'sum',
    'facility_id': 'count'
}).rename(columns={
    'hai_6_numerator': 'Number of observations',
    'facility_id': 'Number of hospitals'
}).assign(**{
    'Percent of cases':lambda x: round(x['Number of observations'] / total_cases * 100, 2),
    'Average cases per hospital':lambda x: round(x['Number of observations'] / x['Number of hospitals'], 2)
}).sort_values(by='Percent of cases', ascending = False)

,Number of observations,Number of hospitals,Percent of cases,Average cases per hospital
hospital_ownership,,,,
Voluntary non-profit - Private,20159,1505,58.39,13.39
Government - Hospital District or Authority,3012,222,8.72,13.57
Voluntary non-profit - Other,2911,261,8.43,11.15
Proprietary,2776,519,8.04,5.35
Voluntary non-profit - Church,2538,229,7.35,11.08
Government - State,1495,35,4.33,42.71
Government - Local,1347,154,3.9,8.75
Department of Defense,150,16,0.43,9.38
Physician,59,15,0.17,3.93


The category with the highest number of observations and hospitals is "Voluntary non-profit - Private", which accounts for almost 60% of all cases, but the highest average number of cases per hospital is observed in the "Government - State" category - around 42.71 cases per facility.  <br><br>
The category with the lowest number of observations and hospitals is "Government - Federal", which accounts for only 0.1% of all cases and only 2.36 cases per hospital.<br>

##### How many cases of infection were documented for private and public hospitals?

In [73]:
private_hospitals = [
    'Proprietary', 'Voluntary non-profit - Private', 'Voluntary non-profit - Church',
    'Voluntary non-profit - Other', 'Physician']

public_hospitals = [
    'Government - Federal', 'Government - State', 'Government - Local',
    'Government - Hospital District or Authority', 'Department of Defense', 'Tribal']

def categorize_hospital(ownership):
    if ownership in private_hospitals:
        return 'Private'
    elif ownership in public_hospitals:
        return 'Public'
    else:
        return 'Unknown'

df_final['ownership_structure'] = df_final['hospital_ownership'].apply(categorize_hospital)

In [74]:
df_final['ownership_structure'] = df_final.ownership_structure.astype('category')

In [75]:
df_final['ownership_structure'].unique().tolist()

['Public', 'Private']

In [76]:
df_final.groupby('ownership_structure')['hai_6_numerator'].sum()

ownership_structure
Private    28443
Public      6080
Name: hai_6_numerator, dtype: Int64

In [77]:
df_final.groupby('ownership_structure')['facility_id'].count()

ownership_structure
Private    2529
Public      444
Name: facility_id, dtype: int64